# Exploring presidential candidates endorsements

In [1]:
import json

## Reading the JSON file

In [2]:
with open('../data/parrainages.json', encoding='utf-8') as f:
    ENDORSEMENTS = json.load(f)

In [3]:
ENDORSEMENTS[0]

{'Civilite': 'M.',
 'Nom': 'CORDIVAL',
 'Prenom': 'Gilles',
 'Mandat': 'Maire',
 'Circonscription': 'Mont-Saint-Père',
 'Departement': 'Aisne',
 'Candidat': 'ARTHAUD Nathalie',
 'DatePublication': '2022-02-01T00:00:00'}

In [4]:
len(ENDORSEMENTS)

13427

## Couting & finding distinct candidates

In [5]:
candidate_names = set()

for endorsment in ENDORSEMENTS:
    candidate_names.add(endorsment['Candidat'])

len(candidate_names)

64

## Filtering candidates that could not run

In France, candidates can only run if they could secure at least 500 endorsements.

In [6]:
endorsements_per_candidate = {}

for endorsement in ENDORSEMENTS:
    candidate_name = endorsement['Candidat']
    
    if candidate_name not in endorsements_per_candidate:
        endorsements_per_candidate[candidate_name] = 0
        
    endorsements_per_candidate[candidate_name] += 1

for candidate_name, count in sorted(endorsements_per_candidate.items(), key=lambda item: item[1], reverse=True)[:20]:
    print(candidate_name, count)

PÉCRESSE Valérie 2636
MACRON Emmanuel 2098
HIDALGO Anne 1440
MÉLENCHON Jean-Luc 906
ZEMMOUR Éric 741
JADOT Yannick 712
LASSALLE Jean 642
ROUSSEL Fabien 626
LE PEN Marine 622
DUPONT-AIGNAN Nicolas 600
POUTOU Philippe 596
ARTHAUD Nathalie 576
ASSELINEAU François 293
TAUBIRA Christiane 274
KAZIB Anasse 160
THOUY Hélène 139
KOENIG Gaspard 107
KUZMANOVIC Georges 49
MIGUET Nicolas 40
EGGER Clara 36


In [7]:
from collections import Counter

In [8]:
Counter()['something']

0

In [9]:
endorsements_per_candidate = Counter()

for endorsement in ENDORSEMENTS:
    endorsements_per_candidate[endorsement['Candidat']] += 1
    
endorsements_per_candidate.most_common(10)

[('PÉCRESSE Valérie', 2636),
 ('MACRON Emmanuel', 2098),
 ('HIDALGO Anne', 1440),
 ('MÉLENCHON Jean-Luc', 906),
 ('ZEMMOUR Éric', 741),
 ('JADOT Yannick', 712),
 ('LASSALLE Jean', 642),
 ('ROUSSEL Fabien', 626),
 ('LE PEN Marine', 622),
 ('DUPONT-AIGNAN Nicolas', 600)]

In [10]:
candidates_able_to_run = set()

for name, count in endorsements_per_candidate.items():
    if count >= 500:
        candidates_able_to_run.add(name)
        
candidates_able_to_run, len(candidates_able_to_run)

({'ARTHAUD Nathalie',
  'DUPONT-AIGNAN Nicolas',
  'HIDALGO Anne',
  'JADOT Yannick',
  'LASSALLE Jean',
  'LE PEN Marine',
  'MACRON Emmanuel',
  'MÉLENCHON Jean-Luc',
  'POUTOU Philippe',
  'PÉCRESSE Valérie',
  'ROUSSEL Fabien',
  'ZEMMOUR Éric'},
 12)

## Keeping only relevant endorsements

We are only interested in endorsements for running candidates and by mayors.

In [11]:
mandates = set()

for endorsement in ENDORSEMENTS:
    mandates.add(endorsement['Mandat'])
    
mandates

{'Conseiller de Paris',
 'Conseiller départemental',
 'Conseiller métropolitain de Lyon',
 'Conseiller régional',
 "Conseiller à l'Assemblée des Français de l'étranger",
 'Conseillère de Paris',
 'Conseillère départementale',
 'Conseillère métropolitaine de Lyon',
 'Conseillère régionale',
 "Conseillère à l'Assemblée des Français de l'étranger",
 'Député',
 'Députée',
 'Maire',
 "Maire d'arrondissement",
 "Maire délégué d'une commune associée ou d'une commune déléguée",
 "Maire déléguée d'une commune associée ou d'une commune déléguée",
 "Membre d'une assemblée d'une collectivité territoriale d'outre-mer à statut particulier",
 "Membre de l'Assemblée de Corse",
 "Président d'un EPCI à fiscalité propre",
 'Président de la Polynésie française',
 'Président du Conseil exécutif de Martinique',
 'Président du conseil consulaire',
 'Président du gouvernement de la Nouvelle-Calédonie',
 "Présidente d'un EPCI à fiscalité propre",
 'Présidente du conseil consulaire',
 'Représentant français au 

In [12]:
RELEVANT_ENDORSEMENTS = []

for endorsement in ENDORSEMENTS:
    if endorsement['Candidat'] in candidates_able_to_run and endorsement['Mandat'].startswith('Maire'):
        RELEVANT_ENDORSEMENTS.append(endorsement)

In [13]:
len(ENDORSEMENTS), len(RELEVANT_ENDORSEMENTS)

(13427, 8117)

## Computing per-candidate gender ratios

In [14]:
genders_per_candidates = {}

for endorsement in RELEVANT_ENDORSEMENTS:
    mayor_is_a_man = endorsement['Civilite'] == 'M.'
    candidate_name = endorsement['Candidat']
    
    if candidate_name not in genders_per_candidates:
        genders_per_candidates[candidate_name] = {'man': 0, 'woman': 0}
        
    gender_key = 'woman'
    
    if mayor_is_a_man:
        gender_key = 'man'
        
    genders_per_candidates[candidate_name][gender_key] += 1
    
genders_per_candidates

{'ARTHAUD Nathalie': {'man': 463, 'woman': 112},
 'DUPONT-AIGNAN Nicolas': {'man': 531, 'woman': 60},
 'HIDALGO Anne': {'man': 565, 'woman': 147},
 'JADOT Yannick': {'man': 253, 'woman': 59},
 'LASSALLE Jean': {'man': 566, 'woman': 57},
 'LE PEN Marine': {'man': 263, 'woman': 50},
 'MACRON Emmanuel': {'man': 1110, 'woman': 186},
 'MÉLENCHON Jean-Luc': {'man': 586, 'woman': 112},
 'PÉCRESSE Valérie': {'man': 1161, 'woman': 212},
 'POUTOU Philippe': {'man': 451, 'woman': 94},
 'ROUSSEL Fabien': {'man': 346, 'woman': 62},
 'ZEMMOUR Éric': {'man': 595, 'woman': 76}}

In [15]:
gender_ratios_per_candidates = {}

for name, counters in genders_per_candidates.items():
    total = counters['woman'] + counters['man']
    ratio = counters['woman'] / total
    gender_ratios_per_candidates[name] = ratio
    
for name, ratio in gender_ratios_per_candidates.items():
    print(name, ratio * 100)

ARTHAUD Nathalie 19.47826086956522
DUPONT-AIGNAN Nicolas 10.152284263959391
HIDALGO Anne 20.646067415730336
JADOT Yannick 18.91025641025641
LASSALLE Jean 9.149277688603531
LE PEN Marine 15.974440894568689
MACRON Emmanuel 14.351851851851851
MÉLENCHON Jean-Luc 16.045845272206304
PÉCRESSE Valérie 15.440640932265111
POUTOU Philippe 17.24770642201835
ROUSSEL Fabien 15.196078431372548
ZEMMOUR Éric 11.326378539493295
